# Library imports



In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import cv2

import os
from tqdm import tqdm

ModuleNotFoundError: No module named 'torch'

# Model Architecture

## Blocks

In [1]:
# Basic Conv Block 정의
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,padding=0):
        super().__init__()

        self.conv = nn.Sequential(
            # TODO : (Conv, BatchNorm, LeakyReLU) 스펙 보고 구현
            nn.Conv2d(in_channels, out_channels, kernel_size, bias=False,stride=stride,padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1)
        )

    def forward(self, x):
        return self.conv(x)


# ResidualBlock 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, num_repeats=1): # residual block은 input channel 수와 output channel 수가 동일하다.
        super().__init__()
        self.num_repeats = num_repeats
        res_layers=[]
        self.residual = nn.Sequential(
            CNNBlock(in_channels,in_channels//2,kernel_size=1,stride=1,padding=0),
            CNNBlock(in_channels//2,in_channels,kernel_size=3,stride=1,padding=1),
        )
        for _ in range(num_repeats):
            res_layers.append(self.residual)
        self.layers = nn.ModuleList(res_layers)


    def forward(self, x):
        for layer in self.layers:
            skip_connection = layer(x)
            x = layer(x)
            x+=skip_connection
        return x



# DarkNet53 정의
class Darknet53(nn.Module):
    def __init__(self,in_channels=3 , num_classes=3):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        # TODO : define darknet53 (위에서 정의한 Conv block과 Res block 활용)
        self.block1 = nn.Sequential(
            CNNBlock(in_channels, 32, kernel_size=3, stride=1, padding=1),
            CNNBlock(32, 64, kernel_size=3, stride=2, padding=1),
            ResidualBlock(64, num_repeats=1),
            CNNBlock(64, 128, kernel_size=3, stride=2, padding=1),
            ResidualBlock(128, num_repeats=2),
            CNNBlock(128, 256, kernel_size=3, stride=2, padding=1),
            ResidualBlock(256, num_repeats=8),
        )
        self.block2 = nn.Sequential(
            CNNBlock(256, 512, kernel_size=3, stride=2, padding=1),
            ResidualBlock(512, num_repeats=8),
        )
        self.block3 = nn.Sequential(
            CNNBlock(512, 1024, kernel_size=3, stride=2, padding=1),
            ResidualBlock(1024, num_repeats=4),
        )

    def forward(self, x):
        # TODO : Darknet53에서 output으로 나오는 세가지 feature map 생산
        high_feature_map = self.block1(x)
        medium_feature_map = self.block2(high_feature_map)
        low_feature_map = self.block3(medium_feature_map)
        return high_feature_map, medium_feature_map, low_feature_map

class UpSampling(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.upsample = nn.Sequential(
            # TODO : YOLO Network Architecture에서 Upsampling에 사용
            CNNBlock(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.Upsample(scale_factor=2)
        )
    def forward(self, x):
        return self.upsample(x)


class YoloBlock(nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.route_conv = nn.Sequential(
            # TODO : define route conv & output conv
            CNNBlock(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            CNNBlock(out_channels, in_channels, kernel_size=3, stride=1, padding=1),
            CNNBlock(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            CNNBlock(out_channels, in_channels, kernel_size=3, stride=1, padding=1),
            CNNBlock(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
        )

    def forward(self, x):
        route = self.route_conv(x)
        return route        #DetectionLayer로 전달


class DetectionLayer(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.num_classes = num_classes
        # TODO : YOLO Network에서 output 된 결과를 이용하여 prediction

        self.pred=nn.Sequential(
            CNNBlock(in_channels, in_channels*2, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels*2,(num_classes+5)*3 , kernel_size=1, bias=False,stride=1,padding=0)
        )

    def forward(self, x):
        output = self.pred(x)
        # TODO : output에 추가적인 처리 필요
        return output

NameError: name 'nn' is not defined

## yolov3 architecture

In [ ]:
class Yolov3(nn.Module):
    def __init__(self, num_classes = 3):
        super().__init__()

        self.num_classes = num_classes

        self.darknet = Darknet53(num_classes=num_classes)

        self.yolo_block_01 = YoloBlock(1024,512)
        self.detectlayer_01 = DetectionLayer(512, num_classes)
        self.upsample_01 = UpSampling(512, 216)

        # input_channels : darknet53 feature map 02 채널(512) + upsampling 채널(256)
        self.yolo_block_02 = YoloBlock(512 + 256, 256)
        self.detectlayer_02 = DetectionLayer(256, num_classes)
        self.upsample_02 = UpSampling(256, 128)

        # input_channels : darknet53 feature map 01 채널(256) + upsampling 채널(128)
        self.yolo_block_03 = YoloBlock(256 + 128, 128)
        self.detectlayer_03 = DetectionLayer(128, num_classes)

    def forward(self, x):
        high_feature_map, medium_feature_map, low_feature_map =self.darknet(x)

        x= self.yolo_block_01(low_feature_map)
        output_01 = self.detectlayer_01(x)
        x = self.upsample_01(x)

        x = self.yolo_block_02(torch.cat([x,medium_feature_map], dim=1))
        output_02 = self.detectlayer_02(x)
        x = self.upsample_02(x)

        x = self.yolo_block_03(torch.cat([x, high_feature_map], dim=1))
        output_03 = self.detectlayer_03(x)

        return output_01, output_02, output_03


# 모델 확인 코드
x = torch.randn((1, 3, 640, 640)) # RGB format의 640 x 640 랜덤 이미지
model = Yolov3(num_classes = 3)
out = model(x)
print(out[0].shape) # torch.Size([1, 3, 13, 13, 8]) / B, RGB, cell size, cell size, (c, x, y, w, h) + classes_prob
print(out[1].shape) # torch.Size([1, 3, 26, 26, 8])
print(out[2].shape) # torch.Size([1, 3, 52, 52, 8])

# torch summary
summary(model, input_size = (2, 3, 640, 640), device = "cpu")

# Define Util & Loss function
참고 자료 : https://www.geeksforgeeks.org/yolov3-from-scratch-using-pytorch/

## Setting anchors

In [ ]:
# Anchors
ANCHORS = [
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)],
]

GRID_SIZE = [13, 26, 52]

## Detection utils

In [ ]:
def iou(box1, box2, is_pred = True):

    # TODO
    iou_score = None

    return iou_score



def nms(bboxes, iou_threshold, threshold):
    # TODO
    bboxes_nms = None
    return bboxes_nms


def convert_cells_to_bboxes():
    # TODO
    converted_bboxes = None
    return converted_bboxes.tolist()


def plot_image(image, boxes):

    plt.show()



## Model checkpoint

In [ ]:
def save_checkpoint(model, optimizer, filename = "dr_bee_checkpoint.ptr.tar"):
    print("==> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)
    
# Function to load checkpoint
def load_checkpoint(checkpoint_file, model, optimizer, lr, device):
    print("==> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr






## Loss function

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self):
        super().__init__()

        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.ce = nn.CrossEntropyLoss()
        self.sigmoid = nn.Sigmoid()


    def forward(self, pred, target, anchors):


        # TODO
        box_loss = 0
        class_loss = 0
        object_loss = 0
        no_object_loss = 0

        return(
            box_loss
            + object_loss
            + no_object_loss
            + class_loss
        )